# Symbolic Computation

In [1]:
import addutils.toc ; addutils.toc.js(ipy_notebook=True)

Symbolic computation (also called computer algebra) emphasizes exact computation where mathematical expressions and objects are represented by algebraic formulas, and not numbers. SymPy is the python module suitable for symbolic computation. See [Sympy Modules Reference](http://docs.sympy.org/dev/modules/index.html).

In [2]:
from addutils import css_notebook
from sympy import *
import numpy as np
import bokeh.plotting as bk
from bokeh.plotting import output_notebook
init_printing(use_latex='mathjax')
css_notebook()

In [3]:
output_notebook()

Loading BokehJS ...

## 1 Basic manipulation

In [5]:
a, b, c = symbols('a, b, c')
exp1 = pi*( a*b*b+2*b*a*b )**c
exp1

          c
  ⎛     2⎞ 
π⋅⎝3⋅a⋅b ⎠ 

We can add assumpions to symbols

In [6]:
k = symbols('k', real=True, positive=True)
k > 0

True

Rational numbers

In [7]:
r1 = Rational(5,6)
r2 = Rational(7,3)
r1+r2

19/6

The expression may be calculated by substituting numerical values to given variables (in this case --> a,b):

In [8]:
exp1.evalf(subs={a:6e-3, b:3, c:2})

0.0824479576008105

We can combine expressions and Numpy arrays

In [9]:
b_vec = np.arange(0, 10, 0.1)
y_vec = np.array([exp1.evalf(subs={a:6e-3, b:bb, c:2}) for bb in b_vec])
y_vec = np.float64(y_vec)
fig = bk.figure(title=None)
fig.line(b_vec, y_vec)
bk.show(fig)

However, this kind of numerical evolution can be very slow, and there is a much more efficient way to do it: Use the function `lambdify` to "compile" a Sympy expression into a function that is much more efficient to evaluate numerically:

In [11]:
f = lambdify([b], exp1, 'numpy')
y_vec = f(b_vec)

## 2 Solve

Algebraically solve equations or systems of equations (expr = 0)

In [12]:
x, y, z = symbols("x y z")
e = (x+y)*40-(z-x)/0.5
e

42.0⋅x + 40⋅y - 2.0⋅z

In [13]:
solve(e)

[{x: -0.952380952380952⋅y + 0.0476190476190476⋅z}]

## 3 Algebra

### 3.1 Expand and Factor

In [14]:
exp2 = (x+1)*sin(x+2)*tanh(x+3)
exp2

(x + 1)⋅sin(x + 2)⋅tanh(x + 3)

In [15]:
expand(exp2)

x⋅sin(x + 2)⋅tanh(x + 3) + sin(x + 2)⋅tanh(x + 3)

The `expand` function takes a number of keywords arguments which we can tell the functions what kind of expansions we want to have performed. For example, to expand trigonometric expressions, use the `trig=True` keyword argument (see `help(sym.expand)` for more info):

In [16]:
expand(exp2, trig=True)

                                      2                                       
-x⋅sin(x)⋅tanh(x + 3) + 2⋅x⋅sin(x)⋅cos (1)⋅tanh(x + 3) + 2⋅x⋅sin(1)⋅cos(1)⋅cos

                                                   2                          
(x)⋅tanh(x + 3) - sin(x)⋅tanh(x + 3) + 2⋅sin(x)⋅cos (1)⋅tanh(x + 3) + 2⋅sin(1)

                          
⋅cos(1)⋅cos(x)⋅tanh(x + 3)

The opposite a product expansion is of course factoring. The factor an expression in SymPy use the `factor` function: 

In [17]:
factor(x**3 + 6 * x**2 + 11*x + 6)

(x + 1)⋅(x + 2)⋅(x + 3)

### 3.2 Symplify

The `simplify` tries to simplify an expression into a nice looking expression, using various techniques. More specific alternatives to the `simplify` functions also exists: `trigsimp`, `powsimp`, `logcombine`, etc. 

The basic usages of these functions are as follows:

In [18]:
simplify(sin(x)**2 + cos(x)**2)

1

In [19]:
simplify(sin(x)/cos(x))

tan(x)

### 3.3 apart and together

In [20]:
exp3 = 1/((a+1)*(a+2))
exp3

       1       
───────────────
(a + 1)⋅(a + 2)

In [19]:
apart(exp3)

    1       1  
- ───── + ─────
  a + 2   a + 1

In [20]:
exp4 = 1/(a+2) + 1/(a+3)
exp4

  1       1  
───── + ─────
a + 3   a + 2

In [21]:
together(exp4)

    2⋅a + 5    
───────────────
(a + 2)⋅(a + 3)

## 4 Calculus

### 4.1 Differentiation and Integration

In [22]:
exp2

(x + 1)⋅sin(x + 2)⋅tanh(x + 3)

In [23]:
diff(exp2)

        ⎛      2           ⎞                                                  
(x + 1)⋅⎝- tanh (x + 3) + 1⎠⋅sin(x + 2) + (x + 1)⋅cos(x + 2)⋅tanh(x + 3) + sin

                   
(x + 2)⋅tanh(x + 3)

In [24]:
exp4

  1       1  
───── + ─────
a + 3   a + 2

In [25]:
integrate(exp3, a)

log(a + 1) - log(a + 2)

In [26]:
integrate(exp(-x**2), (x, -oo, oo))

√π

### 4.2 Sum and Products

In [27]:
n = Symbol("n")
exp5 = 1/n**2
exp5

1 
──
 2
n 

In [28]:
Sum(exp5, (n, 1, oo))

  ∞     
 ____   
 ╲      
  ╲   1 
   ╲  ──
   ╱   2
  ╱   n 
 ╱      
 ‾‾‾‾   
n = 1   

In [29]:
Sum(exp5, (n, 1, oo)).evalf()

1.64493406684823

In [30]:
Product(exp5, (n, 1, oo))

    ∞        
┬────────┬   
│        │ 1 
│        │ ──
│        │  2
│        │ n 
│        │   
  n = 1      

### 4.3 Limits

In [31]:
limit(1/x, x, 0, dir="+")

∞

In [32]:
limit(1/x, x, 0, dir="-")

-∞

### 4.4 Series

By default it expands the expression around $x=0$, but we can expand around any value of $x$ by explicitly include a value in the function call. It's possible to specify to which order the series expansion should be carried out:

In [33]:
exp6 = exp(x)
series(exp6, x)

         2    3    4     5        
        x    x    x     x     ⎛ 6⎞
1 + x + ── + ── + ── + ─── + O⎝x ⎠
        2    6    24   120        

In [34]:
series(exp(x), x, 1, 10)

                         2            3            4            5            6
                ℯ⋅(x - 1)    ℯ⋅(x - 1)    ℯ⋅(x - 1)    ℯ⋅(x - 1)    ℯ⋅(x - 1) 
ℯ + ℯ⋅(x - 1) + ────────── + ────────── + ────────── + ────────── + ──────────
                    2            6            24          120          720    

            7            8            9                      
   ℯ⋅(x - 1)    ℯ⋅(x - 1)    ℯ⋅(x - 1)     ⎛       10       ⎞
 + ────────── + ────────── + ────────── + O⎝(x - 1)  ; x → 1⎠
      5040        40320        362880                        

## 5 Linear algebra

### 5.1 Matrices

In [35]:
m11, m12, m21, m22 = symbols("m11, m12, m21, m22")
b1, b2 = symbols("b1, b2")
A = Matrix([[m11, m12],[m21, m22]])
A

⎡m₁₁  m₁₂⎤
⎢        ⎥
⎣m₂₁  m₂₂⎦

In [36]:
b = Matrix([[b1], [b2]])
b

⎡b₁⎤
⎢  ⎥
⎣b₂⎦

In [37]:
A * b

⎡b₁⋅m₁₁ + b₂⋅m₁₂⎤
⎢               ⎥
⎣b₁⋅m₂₁ + b₂⋅m₂₂⎦

In [38]:
A.det()

m₁₁⋅m₂₂ - m₁₂⋅m₂₁

In [39]:
A.inv()

⎡       m₂₂               -m₁₂       ⎤
⎢─────────────────  ─────────────────⎥
⎢m₁₁⋅m₂₂ - m₁₂⋅m₂₁  m₁₁⋅m₂₂ - m₁₂⋅m₂₁⎥
⎢                                    ⎥
⎢      -m₂₁                m₁₁       ⎥
⎢─────────────────  ─────────────────⎥
⎣m₁₁⋅m₂₂ - m₁₂⋅m₂₁  m₁₁⋅m₂₂ - m₁₂⋅m₂₁⎦

## 6 Solving Equations

In [40]:
exp7 = x**4 - x**2 - a
exp7

      4    2
-a + x  - x 

In [41]:
solve(exp7,x)

⎡      ___________________       ___________________        _________________ 
⎢     ╱     _________           ╱     _________            ╱   _________      
⎢    ╱    ╲╱ 4⋅a + 1    1      ╱    ╲╱ 4⋅a + 1    1       ╱  ╲╱ 4⋅a + 1    1  
⎢-  ╱   - ─────────── + ─ ,   ╱   - ─────────── + ─ , -  ╱   ─────────── + ─ ,
⎣ ╲╱           2        2   ╲╱           2        2    ╲╱         2        2  

      _________________⎤
     ╱   _________     ⎥
    ╱  ╲╱ 4⋅a + 1    1 ⎥
   ╱   ─────────── + ─ ⎥
 ╲╱         2        2 ⎦

---

Visit [www.add-for.com](<http://www.add-for.com/>) for more tutorials and updates.

This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.